In [1]:
from importlib import reload
import backtest
import order_book
import signal_1

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import yfinance as yf
import pickle
import os
import math
from ReferentialData.polygonData import PolygonAPI

In [35]:
# Loading "pairs trade" buckets
DIR = r'C:\Users\raymo\OneDrive\Desktop\Ray Stuff\_Cache'
COR_DIR = r'\Users\raymo\OneDrive\Desktop\Playground\Financial-Modelling-Playground\Quant_Trading\Clustering'
with open(os.path.join(COR_DIR, "correlation_buckets_no_shorts.pkl"), "rb") as file:
    BUCKETS = pickle.load(file)

In [36]:
# Concatenating dictionary value lists
UNIVERSE=[]
for i in list(BUCKETS.values()):
    UNIVERSE.extend(i)

In [37]:
colnames = ["Close", "Volume"]
DataLoader = PolygonAPI()
MID_LOADED = False
HIGH_LOADED = False
if not MID_LOADED:
    mid_df = {}
if not HIGH_LOADED:
    high_df = {}
### loading in low frequency data
if len(mid_df) == 0:
    mid_df = DataLoader.getPrices(UNIVERSE, "day", logDir=os.path.join(DIR,"Beta_Callibration"), _parallel=True)
    MID_LOADED = True

### loading in high frequency data
if len(high_df) == 0:
    high_df = DataLoader.getPrices(UNIVERSE, "minute", logDir=DIR, _parallel=True)
    HIGH_LOADED = True


## Analysis of first correlation bucket

In [38]:
BUCKETS

{'AOA': ['AOA', 'SPXS', 'SPXU', 'SH', 'SDS'],
 'AIQ': ['AIQ', 'SQQQ', 'QID', 'PSQ'],
 'AGQ': ['AGQ', 'ZSL'],
 'AGG': ['AGG', 'TTT', 'TYO', 'TMV', 'TBT', 'TBX', 'PST'],
 'ACWX': ['ACWX', 'EPV', 'EFZ', 'EFU'],
 'ACWI': ['ACWI', 'SPXU', 'SPXS', 'SH', 'SDS'],
 'AAXJ': ['AAXJ', 'EUM', 'EEV'],
 'ZSL': ['ZSL', 'SLV', 'SIVR', 'GLTR'],
 'ZROZ': ['ZROZ', 'TMV', 'TTT', 'TBT'],
 'XBI': ['XBI', 'LABD'],
 'YCL': ['YCL', 'YCS'],
 'YINN': ['YINN', 'YANG', 'YXI', 'FXP'],
 'YCS': ['YCS', 'FXY'],
 'YXI': ['YXI', 'XPP', 'KWEB', 'GXC', 'FXI', 'CWEB', 'BKF'],
 'YANG': ['YANG', 'XPP', 'KWEB', 'GXC', 'FXI', 'CWEB', 'BKF'],
 'XT': ['XT', 'HIBS'],
 'XTR': ['XTR', 'SQQQ', 'SPXS', 'SPXU', 'SH', 'SDS', 'QID', 'PSQ'],
 'XSOE': ['XSOE', 'EUM', 'EEV'],
 'XSVM': ['XSVM', 'TWM', 'TZA', 'SRTY', 'SMDD', 'SDD', 'RWM'],
 'XSMO': ['XSMO', 'TZA', 'TWM', 'SRTY', 'SMDD', 'SDD', 'RWM'],
 'XPP': ['XPP', 'FXP'],
 'XNTK': ['XNTK', 'SQQQ', 'SOXS', 'QID', 'PSQ'],
 'XMLV': ['XMLV', 'SMDD'],
 'XMMO': ['XMMO', 'SMDD'],
 'XLG': ['XLG', 

In [39]:
bucket_mask = lambda price_data: price_data[0] in BUCKETS[list(BUCKETS.keys())[0]]
hf_bucket = dict(filter(bucket_mask, high_df.items()))
mf_bucket = dict(filter(bucket_mask, mid_df.items()))

### Need to find weighting for positions in portfolio

In [40]:
beta = pd.read_csv(r'C:\Users\raymo\OneDrive\Desktop\Ray Stuff\_Cache\Beta_Callibration\^FTW5000_beta.csv')
beta = beta.loc[:, ~beta.columns.str.contains('^Unnamed')]
beta['Beta'] = [float(x.split(",")[0][1:]) for x in beta['Beta']]
beta = beta[beta.apply(bucket_mask, axis=1)]
beta = beta.set_index("Ticker")
beta = beta.to_dict()['Beta']

C:\Users\raymo\AppData\Local\Temp\ipykernel_51272\4011338393.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bucket_mask = lambda price_data: price_data[0] in BUCKETS[list(BUCKETS.keys())[0]]


In [41]:
corr = pd.read_csv(r'C:\Users\raymo\OneDrive\Desktop\Ray Stuff\_Cache\Beta_Callibration\pairwise_corr.csv')
corr = corr.loc[:, ~corr.columns.str.contains('^Unnamed')]
bucket_corr_mask = lambda price_data: len(set(price_data[0].split(',')).intersection(set(BUCKETS[list(BUCKETS.keys())[0]])))==2
corr = corr[corr.apply(bucket_corr_mask, axis=1)]
corr = corr.sort_values(by="Cross Correlation")
corr

C:\Users\raymo\AppData\Local\Temp\ipykernel_51272\4063414317.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bucket_corr_mask = lambda price_data: len(set(price_data[0].split(',')).intersection(set(BUCKETS[list(BUCKETS.keys())[0]])))==2


,Ticker Pair,Cross Correlation
7085,"AOA,SPXS",-0.929647
7087,"AOA,SPXU",-0.928230
7130,"AOA,SDS",-0.927236
7126,"AOA,SH",-0.913910
129151,"SPXS,SH",0.978531
128576,"SPXU,SH",0.990202
129164,"SPXS,SDS",0.992771
152082,"SH,SDS",0.993211
128529,"SPXU,SPXS",0.994507
128584,"SPXU,SDS",0.998731


In [42]:
hf_bucket['AOA']

,Unnamed: 0,open,high,low,close,volume,vwap,timestamp,transactions,otc
0,0,60.2200,60.2200,60.2200,60.2200,3437,60.2233,1672756200000,78,NaN
1,1,60.2900,60.3000,60.2800,60.3000,1187,60.2927,1672756380000,21,NaN
2,2,60.3363,60.3363,60.3363,60.3363,2742,60.3349,1672756440000,4,NaN
3,3,60.3300,60.3300,60.3300,60.3300,295,60.3506,1672756500000,7,NaN
4,4,60.3300,60.3300,60.3300,60.3300,629,60.3392,1672756560000,9,NaN
...,...,...,...,...,...,...,...,...,...,...
11807,11807,65.7850,65.7900,65.7850,65.7900,1306,65.7854,1686686040000,3,NaN
11808,11808,65.7900,65.7900,65.7900,65.7900,640,65.7897,1686686100000,6,NaN
11809,11809,65.7900,65.7900,65.7899,65.7899,747,65.7898,1686686160000,7,NaN
11810,11810,65.7800,65.7800,65.7800,65.7800,643,65.7801,1686686220000,9,NaN


In [44]:
from importlib import reload
import OptimizationEngine.mipEngine
reload(OptimizationEngine.mipEngine)
logReturnTransform = lambda df : np.log(df) - np.log(df.shift(1))

mf_summary = {}
for data in mf_bucket:
    mu = mf_bucket[data][data].mean()
    std = mf_bucket[data][data].std()
    mu_hf = logReturnTransform(hf_bucket[data]['close']).mean()
    std_hf = logReturnTransform(hf_bucket[data]['close']).std()
    mf_summary[data] = {"Mu": mu, "Sigma":std, "Beta":beta[data],
                        "Mu_Min": mu_hf, "Sigma_Min": std_hf}
mf_summary = pd.DataFrame(mf_summary)
bounds = {'Mu':0, 'Sigma':0, 'Beta':0, 'Mu_Min':0, 'Sigma_Min':0}
MIP = OptimizationEngine.mipEngine.mipEngine()
# MIP._addVariables(mf_summary.columns, var_bounds=(None,None), var_type="INTEGER")
MIP._addConstraints(mf_summary, bounds, var_bounds=(1,None))
MIP._optimize()
#pd.DataFrame(mf_summary)

optimal solution cost 0.0 found
solution:


In [34]:
mf_summary

,AVDV,VYMI,VXUS,VSS,VEU,VEA,SPDW,SCZ,SCHF,SCHC,...,IDEV,IEFA,GWX,FNDF,FNDC,FIVA,DLS,DIM,DFIV,DFAX
Mu,0.000378,0.000325,0.000316,0.000250,0.000320,0.000306,0.000321,0.000223,0.000320,0.000242,...,0.000333,0.000318,0.000159,0.000337,0.000265,0.000348,0.000202,0.000253,0.000364,0.000330
Sigma,0.009434,0.008210,0.008429,0.008890,0.008440,0.008652,0.008568,0.009168,0.008508,0.009463,...,0.008608,0.008690,0.009266,0.008545,0.008835,0.008619,0.008969,0.008554,0.008629,0.008251
Beta,-0.003775,-0.004282,-0.003951,-0.002815,-0.004001,-0.003641,-0.003889,-0.002364,-0.003928,-0.002400,...,-0.004003,-0.003747,-0.001645,-0.004107,-0.003014,-0.004165,-0.002235,-0.003073,-0.004348,-0.004313
Mu_Min,0.000003,0.000002,0.000002,0.000003,0.000002,0.000002,0.000003,0.000001,0.000003,0.000003,...,0.000001,0.000002,0.000006,0.000003,0.000005,0.000065,0.000005,0.000041,0.000002,0.000003
Sigma_Min,0.000841,0.000970,0.000691,0.000786,0.000588,0.000642,0.000677,0.000688,0.000992,0.000780,...,0.000879,0.000693,0.001300,0.000744,0.001500,0.002898,0.001048,0.002399,0.000612,0.000585


## Portfolio Optimization

## Grid Search for Strongest Mean Reversion Strategy

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
## analysis between adr spread and volume pct
df = combined[:-30]

df.fillna(method='ffill', inplace=True)
df['ADR_Vol_Pct'] = df['TSMC34_SA_Volume']/(df['TSMC34_SA_Volume'] +df['TSM_Volume'])
mean = df['ADR_Vol_Pct'].mean()
std = df['ADR_Vol_Pct'].std()
df['ADR_Vol_Pct'] = (df['ADR_Vol_Pct'] - mean)/std
df['ADR_Spread'] = df['TSM_Close'] - df['TSMC34_SA_Close']
mean = df['ADR_Spread'].mean()
std = df['ADR_Spread'].std()
df['ADR_Spread'] = (df['ADR_Spread'] - mean)/std
coint_test_result = coint_johansen(df[['ADR_Vol_Pct','ADR_Spread']], det_order=0, k_ar_diff=1)
tracevalues = coint_test_result.lr1
critical_values = coint_test_result.cvt
for i, (stock1, stock2) in enumerate([('ADR_Vol_Pct', 'ADR_Spread')]):
    if (tracevalues[i] > critical_values[:, 1]).all():
        print(f"Pair {i + 1} ({stock1} and {stock2}) is cointegrated.")
    else:
        print(f"Pair {i + 1} ({stock1} and {stock2}) is not cointegrated.")

In [ ]:
import statsmodels.api as sm
# adding the constant term
x = df[['ADR_Spread']].shift(1)[1:]#.tolist()
y = df['ADR_Spread'][1:].tolist()
#x = sm.add_constant(x)
 
# performing the regression
# and fitting the model
result = sm.OLS(y, x).fit()
 
# printing the summary table
print(result.summary())

In [ ]:
result.coef()

In [ ]:

df['ADR_Vol_MA20'] = df['ADR_Vol_Pct'].rolling(20).mean()
df['ADR_1sd_MA20'] = df['ADR_Vol_Pct'].rolling(20).std()
df['primitive_signal'] = df.apply(lambda dr: 1 if dr.ADR_Vol_Pct < dr.ADR_Vol_MA20 - dr.ADR_1sd_MA20 else 0, axis=1).rolling(5).mean()
## add persistence to signal - to optimize
df['Signal'] = [1 if x > 0 else 0 for x in df['primitive_signal']]
df['BuySell'] = df['Signal'] - df['Signal'].shift(1)
df["index"] = pd.to_datetime(df["index"])
df

In [ ]:
BBO = order_book.Book()
for row in df.itertuples():
    if (row.BuySell == 1):
        print('bought spread at ', row.index)
        BBO.addOrder(row.index, "TSM", row.TSM_Close, -100)
        BBO.addOrder(row.index, "TSMC34_SA", row.TSMC34_SA_Close, 150)
    elif (row.BuySell == -1):
        print('sold spread at ', row.index)
        BBO.addOrder(row.index, "TSM", row.TSM_Close, 100)
        BBO.addOrder(row.index, "TSMC34_SA", row.TSMC34_SA_Close, -150)
#BBO.addOrder(df_res.iloc[100]["Date"], "TSM", 300.0, 10)
#BBO.addOrder(df_res.iloc[120]["Date"], "TSM", 250.0, 25)
#BBO.addOrder(df_res.iloc[200]["Date"], "TSM", 350.0, -30)
#BBO.getTickerBook(["TSLA"]).tail(1)#.loc['Quantity']
#BBO.get_history()

#BBO.addOrder(df_res.iloc[50]["Date"], "TSMC34.SA", 300.0, 10)
#BBO.addOrder(df_res.iloc[150]["Date"], "TSMC34.SA", 250.0, -25)

In [ ]:
BBO.getHistoryDf().Date

In [ ]:
df = df.rename(columns={"index":"Date"})
df1 = df[['Date', 'TSMC34_SA_Close']]
df2 = df[['Date', 'TSM_Close']]
df1['Ticker'] = 'TSMC34_SA'
df2['Ticker'] = 'TSM'

df_res = pd.concat([df1.rename(columns={'TSMC34_SA_Close':'Price'}),df2.rename(columns={'TSM_Close':'Price'})])
print(df_res)
#df1.drop('index', inplace=True)
#df_res.rename(columns={'Datetime':'Date'}, inplace=True)
df_res# = df1[['Date', 'Ticker']]
BBO.backfillPnL(df_res)

In [71]:
hist = BBO.getHistoryDf()

In [ ]:
BBO.getTickerBook(['TSM'])

In [ ]:
plt.plot(hist[['PnL']])
plt.show()

In [ ]:
print(hist)

In [ ]:
BBO.get_books()

In [ ]:
hist